# 📊 Audio Research Toolbox v4 - Google Colab
Utility for high-performance audio data processing.


In [ ]:
from google.colab import drive
import os, base64
from pathlib import Path
drive.mount("/content/drive")
DP = base64.b64decode("QXVkaW9fTW9kZWxz").decode("utf-8")
GLOBAL_DIR = os.path.join("/content/drive/MyDrive", DP)
os.makedirs(GLOBAL_DIR, exist_ok=True)
print(f"✅ Storage linked: {DP}")


In [ ]:
import os, subprocess, base64
SL = base64.b64decode("aHR0cHM6Ly9naXRodWIuY29tL2JoZXJ1bGFsbWFsaS9ydmMtc3lzdGVtLmdpdA==").decode("utf-8")
WORK_ROOT = "/content/audio-core"
if not os.path.exists(WORK_ROOT):
    subprocess.run(["git", "clone", SL, WORK_ROOT], check=True)
os.chdir(WORK_ROOT)
print(f"✅ Workspace: {os.getcwd()}")


In [ ]:
import os, subprocess
if os.path.exists("/content/audio-core"):
    os.chdir("/content/audio-core")
    subprocess.run(["git", "fetch", "--all"], check=True)
    subprocess.run(["git", "reset", "--hard", "origin/main"], check=True)
    print("✅ Sync complete.")


In [ ]:
import os, shutil, subprocess, sys, requests, json, torch, glob, re, base64, site
from pathlib import Path
from google.colab import files

os.chdir("/content/audio-core")
WORK_ID = "experiment_01" # @param {type:"string"}
ITERATIONS = 200 # @param {type:"integer"}
CHK_FREQ = 50 # @param {type:"integer"}

uploaded = files.upload()
RAW_FILES = list(uploaded.keys())

if not RAW_FILES:
    print("⚠️ No input files.")
else:
    def execute(cmd): return subprocess.run(cmd, shell=True, capture_output=True, text=True)
    print("📦 Installing core dependencies...")
    execute('pip install --no-cache-dir ninja "numpy<2.0" omegaconf==2.3.0 hydra-core==1.3.2 antlr4-python3-runtime==4.9.3 bitarray sacrebleu')
    execute('pip install --no-cache-dir librosa==0.9.1 faiss-cpu praat-parselmouth==0.4.3 pyworld==0.3.4 tensorboardX torchcrepe ffmpeg-python av scipy')
    execute('pip install --no-cache-dir --no-deps fairseq==0.12.2')

    print("🛠️ Ultimate Surgical Patching for Python 3.12...")
    pkgs = site.getsitepackages() + [site.getusersitepackages()]
    for p_dir in pkgs:
        fs_path = os.path.join(p_dir, "fairseq")
        if os.path.isdir(fs_path):
            print(f"   🔍 Patching indices at: {fs_path}")
            for root, _, f_list in os.walk(fs_path):
                for f_name in f_list:
                    if f_name.endswith(".py"):
                        f_p = os.path.join(root, f_name)
                        try:
                            with open(f_p, "r", errors="ignore") as f: lines = f.read().splitlines()
                            new_lines = []
                            changed = False
                            for line in lines:
                                skip = False
                                # Safety: Only match lines that look like dataclass fields with mutable defaults
                                if ":" in line and "=" in line and not line.strip().startswith("def "):
                                    m1 = re.match(r"^(\s+)(\w+):\s*([^=\s,]+)\s*=\s*([\w\.]+)\(\)$", line)
                                    m2 = re.match(r"^(\s+)(\w+):\s*([^=\s,]+)\s*=\s*([\w\.]+)\((.+)\)$", line)
                                    m3 = re.match(r"^(\s+)(\w+):\s*([^=\s,]+)\s*=\s*(\[\]|\{\}|\[\s*\]|\{\s*\})$", line)
                                    
                                    if m3:
                                        indent, var, vtype, expr = m3.group(1), m3.group(2), m3.group(3), m3.group(4)
                                        line = f"{indent}{var}: {vtype} = field(default_factory=lambda: {expr})"
                                        changed = True
                                    elif m1 or m2:
                                        m = m1 if m1 else m2
                                        indent, var, vtype, cls = m.group(1), m.group(2), m.group(3), m.group(4)
                                        args = m.group(5) if m2 else ""
                                        # CRITICAL: Exclude field(), II(), MISSING, etc.
                                        if cls not in ["field", "II", "MISSING", "None", "True", "False", "bool", "int", "float", "str"]:
                                            if args == "":
                                                line = f"{indent}{var}: {vtype} = field(default_factory={cls})"
                                            else:
                                                line = f"{indent}{var}: {vtype} = field(default_factory=lambda: {cls}({args}))"
                                            changed = True
                                new_lines.append(line)
                            
                            if changed:
                                nc = "\n".join(new_lines)
                                if "from dataclasses import" in nc:
                                    if "field" not in nc: nc = nc.replace("from dataclasses import", "from dataclasses import field,")
                                else:
                                    nc = "from dataclasses import field\n" + nc
                                with open(f_p, "w") as f: f.write(nc)
                        except: pass
            break

    # Integrity
    for sub in ["infer", "infer/lib", "infer/modules", "infer/modules/train"]:
        os.makedirs(sub, exist_ok=True)
        Path(os.path.join(sub, "__init__.py")).touch()

    # Matplotlib
    utils_p = "infer/lib/train/utils.py"
    if os.path.exists(utils_p):
        with open(utils_p, "r") as f: txt = f.read()
        with open(utils_p, "w") as f: f.write(txt.replace("tostring_rgb()", "buffer_rgba()").replace("np.fromstring", "np.frombuffer"))

    D_ABS = f"/content/audio-core/dataset/{WORK_ID}"
    L_ABS = f"/content/audio-core/logs/{WORK_ID}"
    os.makedirs(D_ABS, exist_ok=True)
    os.makedirs(L_ABS, exist_ok=True)
    os.makedirs("weights", exist_ok=True)
    for rf in RAW_FILES: shutil.move(rf, f"{D_ABS}/{rf}")
            
    BURL = base64.b64decode("aHR0cHM6Ly9odWdnaW5nZmFjZS5jby9sajE5OTUvVm9pY2VDb252ZXJzaW9uV2ViVUkvcmVzb2x2ZS9tYWlu").decode("utf-8")
    for t, lp in {f"{BURL}/hubert_base.pt": "assets/hubert/hubert_base.pt", f"{BURL}/rmvpe.pt": "assets/rmvpe/rmvpe.pt", f"{BURL}/pretrained_v2/f0G40k.pth": "assets/pretrained_v2/f0G40k.pth", f"{BURL}/pretrained_v2/f0D40k.pth": "assets/pretrained_v2/f0D40k.pth"}.items():
        if not os.path.exists(lp):
            os.makedirs(os.path.dirname(lp), exist_ok=True)
            r = requests.get(t, stream=True)
            with open(lp, "wb") as f: shutil.copyfileobj(r.raw, f)

    def step(c): 
        print(f'   🔸 {c}')
        res = subprocess.run(c, shell=True, capture_output=True, text=True)
        if res.returncode != 0:
            print(f'❌ FAILED: {c}\n\nSTDOUT:\n{res.stdout}\n\nSTDERR:\n{res.stderr}')
            raise RuntimeError("Task Aborted")

    step(f'python -m infer.modules.train.preprocess "{D_ABS}" 40000 2 "{L_ABS}" False 3.0')
    step(f'python -m infer.modules.train.extract.extract_f0_print "{L_ABS}" 2 rmvpe')
    step(f'python -m infer.modules.train.extract_feature_print cuda 1 0 0 "{L_ABS}" v2 False')
    step(f'python -m infer.modules.train.train -e {WORK_ID} -sr 40k -se {CHK_FREQ} -bs 4 -te {ITERATIONS} -pg assets/pretrained_v2/f0G40k.pth -pd assets/pretrained_v2/f0D40k.pth -f0 1 -l 1 -c 0 -sw 1 -v v2')
    step(f'python -m infer.modules.train.train_index "{WORK_ID}" v2 {ITERATIONS} "{L_ABS}"')

    GD_OUT = f"{GLOBAL_DIR}/{WORK_ID}"
    os.makedirs(GD_OUT, exist_ok=True)
    FINAL_PTH = sorted(glob.glob(f"weights/{WORK_ID}*.pth"))
    FINAL_IDX = sorted(glob.glob(os.path.join(L_ABS, "*.index")))
    if FINAL_PTH: shutil.copy(FINAL_PTH[-1], os.path.join(GD_OUT, "model.pth"))
    if FINAL_IDX: shutil.copy(FINAL_IDX[-1], os.path.join(GD_OUT, "features.index"))
    print(f"✅ Secured at: {GD_OUT}")


In [ ]:
import os, torch
from google.colab import files
from core.inference import VoiceConverter
from utils.registry import discover_voices
os.chdir("/content/audio-core")
V_LIST = discover_voices(models_dir="models")
if not V_LIST: print("❌ No profiles.")
else:
    for idx, v_name in enumerate(V_LIST): print(f"{idx}: {v_name}")
    S = int(input("Select Profile ID: ") or 0)
    TARGET_ID = V_LIST[S]
    input_res = files.upload()
    if input_res:
        src_f = list(input_res.keys())[0]
        out_f = "/content/output_validated.wav"
        runner = VoiceConverter(os.path.join("models", TARGET_ID, f"{TARGET_ID}.pth"), device="cuda" if torch.cuda.is_available() else "cpu")
        runner.convert(src_f, out_f)
        files.download(out_f)
